# Gram-OOD 
Notebook adapted from @paaatcha https://github.com/paaatcha/gram-ood

# install

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-83ab4bbb-d958-4be3-c776-ef36e834772b)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/paaatcha/gram-ood.git

Cloning into 'gram-ood'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 119 (delta 36), reused 107 (delta 29), pack-reused 0
Receiving objects: 100% (119/119), 2.69 MiB | 24.77 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [ ]:
!cp /content/drive/MyDrive/ISIC/checkpoints.zip ./
!unzip checkpoints.zip -d gram-ood/checkpoints/

Archive:  checkpoints.zip
  inflating: gram-ood/checkpoints/densenet-121_checkpoint.pth  
  inflating: gram-ood/checkpoints/mobilenet_checkpoint.pth  
  inflating: gram-ood/checkpoints/resnet-50_checkpoint.pth  
  inflating: gram-ood/checkpoints/vgg-16_checkpoint.pth  


In [ ]:
!cp /content/drive/MyDrive/ISIC/data.zip ./
!unzip data.zip -d gram-ood/

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072931.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072932.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072933.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072935.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072938.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072939.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072940.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072941.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072942.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072951.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072953.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072960.jpeg  
  inflating: gram-ood/data/skin_cancer/train/img/ISIC_0072962.jpeg  
  inflating: gram-ood/data/skin_canc

In [ ]:
!pip install timm
!pip install wandb
!pip install pytorch_lightning
!rm -r sample_data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 181 kB 68.7 MB/s 
     |████████████████████████████████| 158 kB 66.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 157 kB 73.5 MB/s 
     |████████████████████████████████| 157 kB 75.3 MB/s 
     |████████████████████████████████| 157 kB 63.9 MB/s 
     |████████████████████████████████| 157 kB 69.1 MB/s 
     |████████████████████████████████| 157 kB 66.9 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 75.6 MB/s 
     |████████████████████████████████| 156 kB 74.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.

In [ ]:
!pip uninstall albumentations -y
!pip install albumentations==1.0.3

Found existing installation: albumentations 1.2.1
Uninstalling albumentations-1.2.1:
  Successfully uninstalled albumentations-1.2.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 3.7 MB/s 


# pytorch lightning model

## Dataset

In [ ]:
import pytorch_lightning as pl
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
import torchvision
import os
from pathlib import Path
import math
import cv2 
import albumentations as A 
from albumentations.pytorch import ToTensorV2
from skimage.transform import resize


In [ ]:
!cp /content/drive/MyDrive/ISIC/2019/train_outliers.csv ./
train_outliers = pd.read_csv(f'train_outliers.csv').reset_index(drop=True)

unk_paths = pd.read_csv(f'/content/drive/MyDrive/ISIC/2019/unk_paths.csv').reset_index(drop=True)


In [ ]:
from PIL import Image
class Dataset(torch.utils.data.Dataset):
    def __init__(self, imgs, labels, trans=None, augmix = None):
        self.imgs = imgs
        self.labels = labels
        self.trans = trans
        self.augmix = augmix

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, ix):
        if self.augmix is not None:
          img = Image.open(self.imgs[ix])
          img = self.trans(img)
        else:
          img = cv2.imread(self.imgs[ix])
          # if unk_paths.path.isin([self.imgs[ix]]).any() == False:
          #   img = microscope_crop(img)
          # img = center_crop(img, (600,600))
          # img = resize(img, 230, 230)
          # if self.labels[ix] != 7:
          #   img = microscope_crop(img)
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          if self.trans and self.augmix is None:
            img = self.trans(image=img)['image']
            img = torch.tensor(img, dtype=torch.float).permute(2,0,1)
        label = torch.tensor(self.labels[ix], dtype=torch.long)
        return img, label

def microscopy_mask(img, p=1.0):
  # https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/159476
  if random.random() < p:
    circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8),
                        (int((img.shape[0]//2)*1.0), int((img.shape[1]//2)*1.0)),
                        random.randint(img.shape[0]//2 - 10, img.shape[0]//2 + 10),
                        (0, 0, 0),-1)
    mask = circle - 255
    img = np.multiply(img, mask)
  return img

def microscope_crop(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
  # Find contour and sort by contour area
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
  # Find bounding box and extract ROI
  for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    ROI = img[y:y+h, x:x+w]
    break
  return ROI

def center_crop(img, dim):
	""" @ Nannigalaxy GitHub"""
	width, height = img.shape[1], img.shape[0]
	crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
	crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
	mid_x, mid_y = int(width/2), int(height/2)
	cw2, ch2 = int(crop_width/2), int(crop_height/2) 
	crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
	return crop_img

def resize(image, width = None, height = None):
  """@ thewaywewere stackoverflow"""
  dim = None
  (h, w) = image.shape[:2]
  if width is None and height is None:
      return image
  if width is None:
      r = height / float(h)
      dim = (int(w * r), height)
  else:
      r = width / float(w)
      dim = (width, int(h * r))
  resized = cv2.resize(image, dim)
  return resized

In [ ]:
# create_df(dir='/content/2019', val_size =0.1, file_name='train', export_csv=True)
# create_df(dir='/content/2019', val_size =0.1, file_name='train', external_data=True, export_csv=True)

In [ ]:
class DataModule(pl.LightningDataModule):

    def __init__(
            self,
            path='2019',
            file='split',
            subset=0,
            batch_size=32,
            train_trans=None,
            val_trans=None,
            num_workers=4,
            pin_memory=True,
            val_size=0.1,
            augmix = None,
            **kwargs):
        super().__init__()
        self.path = path
        self.file = file
        self.train_trans = train_trans
        self.subset = subset
        self.val_trans = val_trans
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory
        self.val_size = val_size
        self.augmix = augmix
        self.t_df = None

    def setup(self, stage=None):
        # create_df(dir='/content/2019', val_size =self.val_size, file_name='train', export_csv=True)
        # create_df(dir='/content/2019', val_size =self.val_size, file_name='train', external_data=True, export_csv=True)
        # read csv files with imgs names and labels
        train = pd.read_csv(f'{self.path}/train{self.file}.csv')
        self.t_df = train
        val = pd.read_csv(f'{self.path}/val{self.file}.csv')
        print("Training samples: ", len(train))
        print("Validation samples: ", len(val))
        if self.subset:
            _, train = train_test_split(
                train,
                test_size=self.subset,
                shuffle=True,
                stratify=train['target'],
                random_state=42
            )
            print("Training only on", len(train), "samples")
    
        # train dataset
        self.train_ds = Dataset(
            train['path'].values,
            train['target'].values,
            trans = self.train_trans if self.train_trans else None,
            augmix = self.augmix if self.augmix is not None else None
            # trans = A.Compose([
            #     getattr(A, trans)(**params) for trans, params in self.train_trans.items()
            # ]) if self.train_trans else None
            # trans = A.Compose(
            #     [
            #      A.SmallestMaxSize(max_size=224),
            #      A.Transpose(p=0.5),
            #      A.VerticalFlip(p=0.5),
            #      A.HorizontalFlip(p=0.5),
            #      A.RandomBrightness(limit=0.2, p=0.7),
            #      A.RandomContrast(limit=0.2, p=0.7),
            #      A.OneOf([
            #               A.MotionBlur(blur_limit=5),
            #               A.MedianBlur(blur_limit=5),
            #               A.Blur(blur_limit=5),
            #               A.GaussNoise(var_limit=(5.0, 30.0)),
            #               ], p=0.7),
            #      A.OneOf([
            #               A.OpticalDistortion(distort_limit=1.0),
            #               A.GridDistortion(num_steps=5, distort_limit=1.),
            #               A.ElasticTransform(alpha=3),
            #               ], p=0.7),
            #     A.CLAHE(clip_limit=1.0, p=0.5),
            #     A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
            #     A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, interpolation=1, border_mode=0, p=0.85),
            #     A.Cutout(max_h_size=int(224*0.375), max_w_size=int(224*0.375), num_holes=1, p=0.7),
            #     A.CenterCrop(height=224,width = 224),
            #     A.Normalize(mean=(0.6670, 0.5290, 0.5240), std = (0.2239, 0.2037, 0.2155), max_pixel_value = 255)
            #     ])
            # )
        )
        # val dataset
        self.val_ds=Dataset(
            val['path'].values,
            val['target'].values,
            trans = self.val_trans if self.val_trans else None
        )

    def train_dataloader(self):
        # return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.num_workers, sampler=ImbalancedDatasetSampler(self.train_ds, labels=self.t_df['target'].values), pin_memory=self.pin_memory)
        return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True, pin_memory=self.pin_memory)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False, pin_memory=self.pin_memory)

Meta csv

In [ ]:
class DataModule(pl.LightningDataModule):

    def __init__(
            self,
            path='2019',
            file='split',
            subset=0,
            batch_size=32,
            train_trans=None,
            val_trans=None,
            num_workers=4,
            pin_memory=True,
            val_size=0.1,
            augmix = None,
            fold = None,
            **kwargs):
        super().__init__()
        self.path = path
        self.file = file
        self.train_trans = train_trans
        self.subset = subset
        self.val_trans = val_trans
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory
        self.val_size = val_size
        self.augmix = augmix
        self.t_df = None
        self.fold = fold

    def setup(self, stage=None):
        # create_df(dir='/content/2019', val_size =self.val_size, file_name='train', export_csv=True)
        # create_df(dir='/content/2019', val_size =self.val_size, file_name='train', external_data=True, export_csv=True)
        # read csv files with imgs names and labels
        train = pd.read_csv(f'{self.path}/train{self.file}.csv')
        train = train.astype({"age_approx": 'str'})
        if self.fold is not None:
          train_ = train[train['fold'] != self.fold]
          val = train[train['fold'] != self.fold]
          train = train_
          print(f'Fold {self.fold} is validating')
        else:
          self.t_df = train
          val = pd.read_csv(f'{self.path}/val{self.file}.csv')
          val = val.astype({"age_approx": 'str'})
          print("Training samples: ", len(train))
          print("Validation samples: ", len(val))
        if self.subset:
            _, train = train_test_split(
                train,
                test_size=self.subset,
                shuffle=True,
                stratify=train['target'],
                random_state=42
            )
            print("Training only on", len(train), "samples")
        # train dataset
        self.train_ds = Dataset(
            train['path'].values,
            train['target'].values,
            trans = self.train_trans if self.train_trans else None,
            augmix = self.augmix if self.augmix is not None else None)
        # val dataset
        self.val_ds=Dataset(
            val['path'].values,
            val['target'].values,
            trans = self.val_trans if self.val_trans else None
        )

    def train_dataloader(self):
        # return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.num_workers, sampler=ImbalancedDatasetSampler(self.train_ds, labels=self.t_df['target'].values), pin_memory=self.pin_memory)
        return DataLoader(self.train_ds, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True, pin_memory=self.pin_memory)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False, pin_memory=self.pin_memory)

## Model

In [ ]:
class FocalLoss(torch.nn.Module):
    def __init__(self, num_classes,weight = None):
        # include focal-loss and label smmooth
        super(FocalLoss, self).__init__()

        self.num_classes = num_classes
        self.weight = weight
        self.softmax = torch.nn.Softmax(dim=1)

        # no weight no fl
        if self.weight is None:
            self.weight = torch.ones(num_classes,dtype = torch.float32)

            self.fl_gamma =  torch.zeros(num_classes,dtype = torch.float32)
            
        else:
            # it's numpy
            # for weight<1.0 est gamma
            self.fl_gamma =  torch.zeros(num_classes,dtype = torch.float32)
            for idx, ww in enumerate(self.weight):
                if ww<=1:
                    fl_gamma = np.floor(-np.log2(ww))
                    self.fl_gamma[idx] = fl_gamma
                    if fl_gamma>0:
                        self.weight[idx] = self.weight[idx] *pow(2.0,fl_gamma/2.0)
            self.weight = torch.from_numpy(self.weight).float()

        self.weight = self.weight.cuda()
        self.fl_gamma = self.fl_gamma.cuda()
        
    def forward(self, pred, targ):
#        if self.use_gpu: 
#            targ = targ.cuda()
        probs = self.softmax(pred)
        #targets_onehot = torch.zeros_like(log_probs).scatter_(1, targ[:,None], 1)

        targ_prob = torch.gather(input = probs,dim=1,index = targ[:,None]).squeeze(1)

        
        fl_gamma = self.fl_gamma.index_select(dim = 0, index = targ)
        fl_weight = self.weight.index_select(dim = 0, index = targ)
        
        
        return torch.mean(-fl_weight * (1.0 - targ_prob+0.001).pow(fl_gamma) * torch.log(targ_prob + 0.001))

In [ ]:
"""
# This python file is implemented eloss functions
@author: Md Mostafa Kamal Sarker
@ email: m.kamal.sarker@gmail.com
@ Date: 23.05.2017
"""

import numpy as np
import torch
import torch.nn.functional as F


def focal_loss(labels, logits, alpha, gamma):
    """Compute the focal loss between `logits` and the ground truth `labels`.
    Focal loss = -alpha_t * (1-pt)^gamma * log(pt)
    where pt is the probability of being classified to the true class.
    pt = p (if true class), otherwise pt = 1 - p. p = sigmoid(logit).
    Args:
      labels: A float tensor of size [batch, num_classes].
      logits: A float tensor of size [batch, num_classes].
      alpha: A float tensor of size [batch_size]
        specifying per-example weight for balanced cross entropy.
      gamma: A float scalar modulating loss from hard and easy examples.
    Returns:
      focal_loss: A float32 scalar representing normalized total loss.
    """    
    BCLoss = F.binary_cross_entropy_with_logits(input = logits, target = labels,reduction = "none")

    if gamma == 0.0:
        modulator = 1.0
    else:
        modulator = torch.exp(-gamma * labels * logits - gamma * torch.log(1 + 
            torch.exp(-1.0 * logits)))

    loss = modulator * BCLoss

    weighted_loss = alpha * loss
    focal_loss = torch.sum(weighted_loss)

    focal_loss /= torch.sum(labels)
    return focal_loss


def CB_loss(logits, labels, samples_per_cls, no_of_classes, loss_type, beta, gamma):
    """Compute the Class Balanced Loss between `logits` and the ground truth `labels`.
    Class Balanced Loss: ((1-beta)/(1-beta^n))*Loss(labels, logits)
    where Loss is one of the standard losses used for Neural Networks.
    Args:
      labels: A int tensor of size [batch].
      logits: A float tensor of size [batch, no_of_classes].
      samples_per_cls: A python list of size [no_of_classes].
      no_of_classes: total number of classes. int
      loss_type: string. One of "sigmoid", "focal", "softmax".
      beta: float. Hyperparameter for Class balanced loss.
      gamma: float. Hyperparameter for Focal loss.
    Returns:
      cb_loss: A float tensor representing class balanced loss
    """
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * no_of_classes

    labels_one_hot = F.one_hot(labels, no_of_classes).float()

    weights = torch.tensor(weights).float().cuda()
    weights = weights.unsqueeze(0)
    weights = weights.repeat(labels_one_hot.shape[0],1) * labels_one_hot
    weights = weights.sum(1)
    weights = weights.unsqueeze(1)
    weights = weights.repeat(1,no_of_classes)

    if loss_type == "focal":
        cb_loss = focal_loss(labels_one_hot, logits, weights, gamma)
    elif loss_type == "sigmoid":
        cb_loss = F.binary_cross_entropy_with_logits(input = logits,target = labels_one_hot, weights = weights)
    elif loss_type == "softmax":
        pred = logits.softmax(dim = 1)
        cb_loss = F.binary_cross_entropy(input = pred, target = labels_one_hot, weight = weights)
    return cb_loss

In [ ]:
class SAM(torch.optim.Optimizer):
    ''' @davda54 https://github.com/davda54/sam '''
    def __init__(self, params, base_optimizer, rho=2.0, adaptive=True, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                self.state[p]["old_p"] = p.data.clone()
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.data = self.state[p]["old_p"]  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

    def load_state_dict(self, state_dict):
        super().load_state_dict(state_dict)
        self.base_optimizer.param_groups = self.param_groups

In [ ]:
import pytorch_lightning as pl
import torchvision
import torch.nn.functional as F
from torchmetrics.functional import accuracy
import torch
from torchvision import transforms
import timm
from torchmetrics import Accuracy, F1Score, Specificity, Precision, Recall, AUROC
from sklearn.metrics import balanced_accuracy_score
import warnings
warnings.filterwarnings('ignore') 
import timm.optim
import wandb
import torch.optim.lr_scheduler


class BaseModel(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        # self.automatic_optimization = False
        self.save_hyperparameters(config)
        num_class = 8
        class_names = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']
        if config['external_data']:
          num_class=9
          class_names = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'UNK', 'VASC']
        self.num_class=num_class
        self.class_names=class_names
        # self.balanced_accuracy = Accuracy(num_classes=num_class, average='macro')
        self.f1 = F1Score(num_classes=num_class, average='macro')
        self.specificity = Specificity(num_classes=num_class, average='macro')
        self.precision_ = Precision(num_classes=num_class, average='macro')
        self.recall = Recall(num_classes=num_class, average='macro')
        self.auc = AUROC(num_classes=num_class)
        self.focal_loss = FocalLoss(num_class)
        self.learning_rate= self.hparams.lr
        
        # self.val_outputs = torch.tensor([]).cpu()
        # self.val_targets = torch.tensor([]).cpu()
        # self.val_outputs = torch.cat([self.val_outputs, y_hat.detach().cpu()])
        # self.val_targets = torch.cat([self.val_targets, y.detach().cpu()])

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        y_hat = self(x)
        # if self.hparams.loss_weight:
        # val_loss = F.cross_entropy(y_hat, y, weight=self.hparams.class_weights)
        # else:
        val_loss = F.cross_entropy(y_hat, y)
        # val_loss = self.focal_loss(y_hat, y)
        # val_loss =  CB_loss(y_hat, y, [867, 3404, 2762, 276, 4914, 13832, 628, 5990, 282], 9, 'focal', beta=0.9999, gamma=0.5)
        # sch = self.lr_schedulers()
        # sch.step(val_loss)

        val_acc = accuracy(y_hat, y)
        val_bacc = balanced_accuracy_score(y.detach().cpu().numpy(), torch.argmax(y_hat.detach().cpu(), dim=1).numpy())
        # val_bacc = self.balanced_accuracy(y_hat, y)
        val_f1 = self.f1(y_hat, y)
        val_specificity = self.specificity(y_hat, y)
        val_precision = self.precision_(y_hat, y)
        val_recall = self.recall(y_hat, y)
        val_auc = self.auc(y_hat, y)
        
        self.log('val_loss', val_loss, prog_bar=True)
        self.log('val_acc', val_acc, prog_bar=True)
        self.log('val_bacc', val_bacc, prog_bar=True)
        self.log('val_f1', val_f1)
        self.log('val_specificity', val_specificity)
        self.log('val_precision', val_precision)
        self.log('val_recall', val_recall)
        self.log('val_auc', val_auc)

        return {"y_hat": y_hat.detach().cpu(), "y": y.detach().cpu()}

    def configure_optimizers(self):
        if 'SAM' == self.hparams.optimizer:
          print('SAM')
          optimizer = SAM(self.parameters(), timm.optim.AdamP, lr=self.hparams.lr)
        elif 'AdamP' == self.hparams.optimizer:
          print('AdamP')
          optimizer = getattr(timm.optim, self.hparams.optimizer)(self.parameters(), lr=self.hparams.lr)  
        elif 'SGD' == self.hparams.optimizer:
          print('SGD')
          optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams.lr, momentum=0.9, weight_decay =5e-04)
        else:
          print('Else')
          optimizer = getattr(torch.optim, self.hparams.optimizer)(self.parameters(), lr=self.hparams.lr)
        if 'scheduler' in self.hparams:
            scheduler = [(scheduler, params) for scheduler, params in self.hparams.scheduler.items()]
            # scheduler = getattr(timm.scheduler.plateau_lr, scheduler[0][0])(optimizer, **scheduler[0][1])
            
    
            scheduler = getattr(torch.optim.lr_scheduler, scheduler[0][0])(optimizer, **scheduler[0][1])
            # scheduler = getattr(torch.optim.lr_scheduler, scheduler[0][0])(optimizer, **scheduler[0][1])
            # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=0, factor=0.8)
            return [optimizer], [scheduler]
            # return [optimizer], [{"scheduler": scheduler}]
        return optimizer

# list(df.target.value_counts().sort_index())

class ViT(BaseModel):
    def __init__(self, config):
        super().__init__(config)
        # self.model = timm.create_model(self.hparams.backbone, img_size=224, pretrained=self.hparams.pretrained, num_classes=9)
        self.model = timm.create_model(self.hparams.backbone,pretrained=self.hparams.pretrained, num_classes=9)
    
    def forward(self, x):
        return self.model(x)

    def compute_loss(self, batch):
        x, y = batch
        return F.cross_entropy(self(x), y, weight=self.hparams.class_weights)

    def training_step(self, batch, batch_idx):
        # optimizer = self.optimizers()
        # scheduler = self.lr_schedulers()
        x, y = batch
        x, y = x.cuda(), y.cuda()
        y_hat = self(x)
        if self.hparams.optimizer == 'SAM':
          # first forward-backward pass
          loss = self.compute_loss(batch)
          self.manual_backward(loss)
          optimizer.first_step(zero_grad=True)
          # second forward-backward pass
          loss_2 = self.compute_loss(batch)
          self.manual_backward(loss_2)
          optimizer.second_step(zero_grad=True)
        else:
          # optimizer.zero_grad()
          # loss = self.compute_loss(batch)
          # loss = self.focal_loss(y_hat, y)
          loss = F.cross_entropy(y_hat, y, weight=self.hparams.class_weights)
          # loss =  CB_loss(y_hat, y, [867, 3404, 2762, 276, 4914, 13832, 628, 5990, 282], 9, 'focal', beta=0.9999, gamma=0.5)
          # self.manual_backward(loss)
          # optimizer.step()
        # step every N epochs
        # if self.trainer.is_last_batch and (self.trainer.current_epoch + 1) % 1 == 0:
        #   scheduler.step()

        acc = accuracy(y_hat, y)
        # train_bacc = self.balanced_accuracy(y_hat, y)
        train_bacc = balanced_accuracy_score(y.detach().cpu().numpy(), torch.argmax(y_hat.detach().cpu(), dim=1).numpy())
        # train_f1 = self.f1(y_hat, y)
        # train_specificity = self.specificity(y_hat, y)
        # train_precision = self.precision_(y_hat, y)
        # train_recall = self.recall(y_hat, y)
        # train_auc = self.auc(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        self.log('train_bacc', train_bacc, prog_bar=True)
        # self.log('train_f1', train_f1, on_step=False, on_epoch=True)
        # self.log('train_specificity', train_specificity, on_step=False, on_epoch=True)
        # self.log('train_precision', train_precision, on_step=False, on_epoch=True)
        # self.log('train_recall', train_recall, on_step=False, on_epoch=True)
        # self.log('train_auc', train_auc, on_step=False, on_epoch=True)

        return loss

    def validation_epoch_end(self, validation_step_outputs):
        all_yhat = torch.cat([x['y_hat'] for x in validation_step_outputs])
        all_y = torch.cat([x['y'] for x in validation_step_outputs])
        self.logger.experiment.log({"val_roc" : wandb.plot.roc_curve(all_y.numpy(), all_yhat.numpy(), self.class_names)})
        self.logger.experiment.log({"val_confusion_matrix" : wandb.plot.confusion_matrix(y_true=all_y.numpy(), preds=torch.argmax(all_yhat, dim=1).numpy(),class_names=self.class_names)})

class Model(BaseModel):

    def __init__(self, config):
        super().__init__(config)
        m = timm.create_model(
            self.hparams.backbone, 
            pretrained=self.hparams.pretrained, 
            features_only=True
        )
        # ct=0
        # for child in m.children():
        #   ct += 1
        #   if ct < 4:
        #     for param in child.parameters():
        #       param.requires_grad = False
        self.backbone = m
        self.head = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(output_size=(1,1)),
            torch.nn.Flatten(),
            torch.nn.Linear(self.backbone.feature_info.channels(-1), self.num_class)
        )

    def forward(self, x):
        features = self.backbone(x)
        return self.head(features[-1])

    def extract_features(self, x):
        # with torch.no_grad():
        #   features = self.backbone(x)
        if self.trainer.current_epoch < self.hparams.unfreeze:
            with torch.no_grad():
                features = self.backbone(x)
        else: 
            features = self.backbone(x)
        return features

    def training_step(self, batch, batch_idx):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        features = self.extract_features(x)
        y_hat = self.head(features[-1])
        loss = F.cross_entropy(y_hat, y, weight=self.hparams.class_weights)
        # loss = self.focal_loss(y_hat, y)
        acc = accuracy(y_hat, y)
        train_bacc = balanced_accuracy_score(y.detach().cpu().numpy(), torch.argmax(y_hat.detach().cpu(), dim=1).numpy())
        # train_f1 = self.f1(y_hat, y)
        # train_specificity = self.specificity(y_hat, y)
        # train_precision = self.precision_(y_hat, y)
        # train_recall = self.recall(y_hat, y)
        # train_auc = self.auc(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        self.log('train_bacc', train_bacc, prog_bar=True)
        # self.log('train_f1', train_f1, prog_bar=True, on_step=False, on_epoch=True)
        # self.log('train_specificity', train_specificity, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_precision', train_precision, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_recall', train_recall, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_auc', train_auc, prog_bar=False, on_step=False, on_epoch=True)
        
        return loss

class Model2(BaseModel):
    ''' Multi-Sample Dropout for Accelerated Training and Better Generalization https://arxiv.org/pdf/1905.09788.pdf'''
    def __init__(self, config):
        super().__init__(config)
        self.m = timm.create_model(
            self.hparams.backbone, 
            pretrained=self.hparams.pretrained, 
            num_classes=0
        )
        self.dropouts = torch.nn.ModuleList([torch.nn.Dropout(0.2) for _ in range(16)])
        self.fc = torch.nn.Linear(2048, self.num_class)

    def forward(self, x):
        x = self.m(x)
        for i,dropout in enumerate(self.dropouts):
            if i== 0:
                out = dropout(x.clone())
                out = self.fc(out)
            else:
                temp_out = dropout(x.clone())
                out += self.fc(temp_out)
        return out/len(self.dropouts)


    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y, weight=self.hparams.class_weights)
        # loss = self.focal_loss(y_hat, y)
        acc = accuracy(y_hat, y)
        train_bacc = balanced_accuracy_score(y.detach().cpu().numpy(), torch.argmax(y_hat.detach().cpu(), dim=1).numpy())
        # train_f1 = self.f1(y_hat, y)
        # train_specificity = self.specificity(y_hat, y)
        # train_precision = self.precision_(y_hat, y)
        # train_recall = self.recall(y_hat, y)
        # train_auc = self.auc(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        self.log('train_bacc', train_bacc, prog_bar=True)
        # self.log('train_f1', train_f1, prog_bar=True, on_step=False, on_epoch=True)
        # self.log('train_specificity', train_specificity, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_precision', train_precision, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_recall', train_recall, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_auc', train_auc, prog_bar=False, on_step=False, on_epoch=True)
        
        return loss

In [ ]:
# m = timm.create_model(
#             'tf_efficientnet_b5_ns', 
#             pretrained=True, 
#             features_only=True
#         )

# ct=0
# for child in m.children():
#   ct += 1
#   if ct < 4:
#     for param in child.parameters():
#       param.requires_grad = False  

# for param in m.parameters():  
#   print(param.requires_grad)

## Model Meta

In [ ]:
import pytorch_lightning as pl
import torchvision
import torch.nn.functional as F
from torchmetrics.functional import accuracy
import torch
from torchvision import transforms
import timm
from torchmetrics import Accuracy, F1Score, Specificity, Precision, Recall, AUROC
from sklearn.metrics import balanced_accuracy_score
import warnings
warnings.filterwarnings('ignore') 
import timm.optim
import wandb
import torch.optim.lr_scheduler


class BaseModel(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        # self.automatic_optimization = False
        self.save_hyperparameters(config)
        num_class = 8
        class_names = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']
        if config['external_data']:
          num_class=9
          class_names = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'UNK', 'VASC']
        self.num_class=num_class
        self.class_names=class_names
        # self.balanced_accuracy = Accuracy(num_classes=num_class, average='macro')
        self.f1 = F1Score(num_classes=num_class, average='macro')
        self.specificity = Specificity(num_classes=num_class, average='macro')
        self.precision_ = Precision(num_classes=num_class, average='macro')
        self.recall = Recall(num_classes=num_class, average='macro')
        self.auc = AUROC(num_classes=num_class)
        # self.focal_loss = FocalLoss(num_class)
        self.learning_rate= self.hparams.lr
        
        # self.val_outputs = torch.tensor([]).cpu()
        # self.val_targets = torch.tensor([]).cpu()
        # self.val_outputs = torch.cat([self.val_outputs, y_hat.detach().cpu()])
        # self.val_targets = torch.cat([self.val_targets, y.detach().cpu()])

    def validation_step(self, batch, batch_idx):
        x, m, y = batch
        y_hat = self(x, m)
        # if self.hparams.loss_weight:
        # val_loss = F.cross_entropy(y_hat, y, weight=self.hparams.class_weights)
        # else:
        val_loss = F.cross_entropy(y_hat, y)
        # val_loss = self.focal_loss(y_hat, y)
        # val_loss =  CB_loss(y_hat, y, [867, 3404, 2762, 276, 4914, 13832, 628, 5990, 282], 9, 'focal', beta=0.9999, gamma=0.5)
        # sch = self.lr_schedulers()
        # sch.step(val_loss)

        val_acc = accuracy(y_hat, y)
        val_bacc = balanced_accuracy_score(y.detach().cpu().numpy(), torch.argmax(y_hat.detach().cpu(), dim=1).numpy())
        # val_bacc = self.balanced_accuracy(y_hat, y)
        val_f1 = self.f1(y_hat, y)
        val_specificity = self.specificity(y_hat, y)
        val_precision = self.precision_(y_hat, y)
        val_recall = self.recall(y_hat, y)
        val_auc = self.auc(y_hat, y)
        
        self.log('val_loss', val_loss, prog_bar=True)
        self.log('val_acc', val_acc, prog_bar=True)
        self.log('val_bacc', val_bacc, prog_bar=True)
        self.log('val_f1', val_f1)
        self.log('val_specificity', val_specificity)
        self.log('val_precision', val_precision)
        self.log('val_recall', val_recall)
        self.log('val_auc', val_auc)

        return {"y_hat": y_hat.detach().cpu(), "y": y.detach().cpu()}

    def configure_optimizers(self):
        if 'SAM' == self.hparams.optimizer:
          print('SAM')
          optimizer = SAM(self.parameters(), timm.optim.AdamP, lr=self.hparams.lr)
        elif 'AdamP' == self.hparams.optimizer:
          print('AdamP')
          optimizer = getattr(timm.optim, self.hparams.optimizer)(self.parameters(), lr=self.hparams.lr)  
        elif 'SGD' == self.hparams.optimizer:
          print('SGD')
          optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams.lr, momentum=0.9, weight_decay =5e-04)
        else:
          print('Else')
          optimizer = getattr(torch.optim, self.hparams.optimizer)(self.parameters(), lr=self.hparams.lr)
        if 'scheduler' in self.hparams:
            scheduler = [(scheduler, params) for scheduler, params in self.hparams.scheduler.items()]
            # scheduler = getattr(timm.scheduler.plateau_lr, scheduler[0][0])(optimizer, **scheduler[0][1])
            
    
            scheduler = getattr(torch.optim.lr_scheduler, scheduler[0][0])(optimizer, **scheduler[0][1])
            # scheduler = getattr(torch.optim.lr_scheduler, scheduler[0][0])(optimizer, **scheduler[0][1])
            # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=0, factor=0.8)
            return [optimizer], [scheduler]
            # return [optimizer], [{"scheduler": scheduler}]
        return optimizer

# list(df.target.value_counts().sort_index())

class ViT(BaseModel):
    def __init__(self, config):
        super().__init__(config)
        # self.model = timm.create_model(self.hparams.backbone, img_size=224, pretrained=self.hparams.pretrained, num_classes=9)
        self.model = timm.create_model(self.hparams.backbone,pretrained=self.hparams.pretrained, num_classes=9)
    
    # def forward(self, x):
    #     return self.model(x)
    def forward(self, x, meta):        
        x = self.backbone(x)
        # get meta
        meta = self.meta_model(meta)
        # concatenate both
        z = torch.cat([x,meta],1)
        # pass to backbone
        return self.head(z)

    def compute_loss(self, batch):
        x, y = batch
        return F.cross_entropy(self(x), y, weight=self.hparams.class_weights)

    def training_step(self, batch, batch_idx):
        # optimizer = self.optimizers()
        # scheduler = self.lr_schedulers()
        x, y = batch
        y_hat = self(x)
        if self.hparams.optimizer == 'SAM':
          # first forward-backward pass
          loss = self.compute_loss(batch)
          self.manual_backward(loss)
          optimizer.first_step(zero_grad=True)
          # second forward-backward pass
          loss_2 = self.compute_loss(batch)
          self.manual_backward(loss_2)
          optimizer.second_step(zero_grad=True)
        else:
          # optimizer.zero_grad()
          # loss = self.compute_loss(batch)
          # loss = self.focal_loss(y_hat, y)
          loss = F.cross_entropy(y_hat, y, weight=self.hparams.class_weights)
          # loss =  CB_loss(y_hat, y, [867, 3404, 2762, 276, 4914, 13832, 628, 5990, 282], 9, 'focal', beta=0.9999, gamma=0.5)
          # self.manual_backward(loss)
          # optimizer.step()
        # step every N epochs
        # if self.trainer.is_last_batch and (self.trainer.current_epoch + 1) % 1 == 0:
        #   scheduler.step()

        acc = accuracy(y_hat, y)
        # train_bacc = self.balanced_accuracy(y_hat, y)
        train_bacc = balanced_accuracy_score(y.detach().cpu().numpy(), torch.argmax(y_hat.detach().cpu(), dim=1).numpy())
        # train_f1 = self.f1(y_hat, y)
        # train_specificity = self.specificity(y_hat, y)
        # train_precision = self.precision_(y_hat, y)
        # train_recall = self.recall(y_hat, y)
        # train_auc = self.auc(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        self.log('train_bacc', train_bacc, prog_bar=True)
        # self.log('train_f1', train_f1, on_step=False, on_epoch=True)
        # self.log('train_specificity', train_specificity, on_step=False, on_epoch=True)
        # self.log('train_precision', train_precision, on_step=False, on_epoch=True)
        # self.log('train_recall', train_recall, on_step=False, on_epoch=True)
        # self.log('train_auc', train_auc, on_step=False, on_epoch=True)

        return loss

    def validation_epoch_end(self, validation_step_outputs):
        all_yhat = torch.cat([x['y_hat'] for x in validation_step_outputs])
        all_y = torch.cat([x['y'] for x in validation_step_outputs])
        self.logger.experiment.log({"val_roc" : wandb.plot.roc_curve(all_y.numpy(), all_yhat.numpy(), self.class_names)})
        self.logger.experiment.log({"val_confusion_matrix" : wandb.plot.confusion_matrix(y_true=all_y.numpy(), preds=torch.argmax(all_yhat, dim=1).numpy(),class_names=self.class_names)})

class Model(BaseModel):

    def __init__(self, config):
        super().__init__(config)
        # meta enconder
        n_features = timm.create_model(self.hparams.backbone,pretrained=self.hparams.pretrained, num_classes = self.num_class).get_classifier().in_features
        self.backbone = timm.create_model(self.hparams.backbone,pretrained=self.hparams.pretrained, num_classes = 0)

        # self.meta_model = torch.nn.Sequential(
        #     torch.nn.Linear(32, 512, bias=True),
        #     torch.nn.BatchNorm1d(512),
        #     torch.nn.Linear(512, 128, bias=True),
        #     torch.nn.BatchNorm1d(128),
        #     torch.nn.ReLU(inplace=True),
        # ) .580 thrs

        self.meta_model = torch.nn.Sequential(
            torch.nn.Linear(32, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(512, 128),
            torch.nn.BatchNorm1d(128),
        )
        # Backbone | head
        #(64x512 and 32x128)
        self.head = torch.nn.Linear(n_features+128, self.num_class, bias=True)  
        # self.head = torch.nn.Sequential(
        #     torch.nn.AdaptiveAvgPool2d(output_size=(1,1)),
        #     torch.nn.Flatten(),
        #     torch.nn.Linear(n_features+128, self.num_class, bias=True)
        # )
        

    def forward(self, x, meta):        
        x = self.backbone(x)
        # get meta
        meta = self.meta_model(meta)
        # concatenate both
        z = torch.cat([x,meta],1)
        # pass to backbone
        return self.head(z)

    def training_step(self, batch, batch_idx):
        x, m, y = batch
        y_hat = self(x,m)
        loss = F.cross_entropy(y_hat, y, weight=self.hparams.class_weights)
        # loss = self.focal_loss(y_hat, y)
        acc = accuracy(y_hat, y)
        train_bacc = balanced_accuracy_score(y.detach().cpu().numpy(), torch.argmax(y_hat.detach().cpu(), dim=1).numpy())
        # train_f1 = self.f1(y_hat, y)
        # train_specificity = self.specificity(y_hat, y)
        # train_precision = self.precision_(y_hat, y)
        # train_recall = self.recall(y_hat, y)
        # train_auc = self.auc(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        self.log('train_bacc', train_bacc, prog_bar=True)
        # self.log('train_f1', train_f1, prog_bar=True, on_step=False, on_epoch=True)
        # self.log('train_specificity', train_specificity, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_precision', train_precision, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_recall', train_recall, prog_bar=False, on_step=False, on_epoch=True)
        # self.log('train_auc', train_auc, prog_bar=False, on_step=False, on_epoch=True)
        
        return loss
        
    def validation_epoch_end(self, validation_step_outputs):
        all_yhat = torch.cat([x['y_hat'] for x in validation_step_outputs])
        all_y = torch.cat([x['y'] for x in validation_step_outputs])
        self.logger.experiment.log({"val_roc" : wandb.plot.roc_curve(all_y.numpy(), all_yhat.numpy(), self.class_names)})
        self.logger.experiment.log({"val_confusion_matrix" : wandb.plot.confusion_matrix(y_true=all_y.numpy(), preds=torch.argmax(all_yhat, dim=1).numpy(),class_names=self.class_names)})


In [ ]:
# model = Model(config)
# output = model(torch.randn(2,3, 456,456), torch.randn(2,32))
# output.shape

In [ ]:
# import timm
# model_arch = 'tf_efficientnet_b5_ns'
# pretrained=True
# n_class=9
# n_features = timm.create_model(model_arch, pretrained=pretrained, num_classes = n_class).get_classifier().in_features
# model = timm.create_model(model_arch, pretrained=pretrained, num_classes = 0)
        

In [ ]:
# backbone = timm.create_model(
#             model_arch, 
#             pretrained=True, 
#             features_only=True
#         )

In [ ]:
# backbone.feature_info.channels(-1)

In [ ]:
# backbone

In [ ]:
# n_features

In [ ]:
# model

# Load model

In [ ]:
import wandb
import albumentations as A
import torchvision
import torchvision.transforms as transforms
from timm.data.auto_augment import rand_augment_transform, augment_and_mix_transform, auto_augment_transform
import albumentations as A
run = wandb.init()
# model_name = 'model-24yj71ho:v9' # rexnext 4d
model_name = 'model-25hhilx3:v14' # efficientnet_b5
#model_name = 'model-rghgy5v3:v5' # efficientnet_b4

artifact = run.use_artifact(f'skin-lesson/skin-lession/{model_name}', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: blobquiet (skin-lesson). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-25hhilx3:v14, 202.03MB. 1 files... Done. 0:0:0.1


In [ ]:
torch_model = ViT.load_from_checkpoint(checkpoint_path=f'/content/artifacts/{model_name}/model.ckpt')
torch_model.eval()
torch_model.cuda()
torch_model.hparams

"backbone":      tf_efficientnet_b4_ns
"batch_size":    16
"class_weights": None
"external_data": True
"loss_weight":   0
"lr":            1e-08
"max_epochs":    20
"num_workers":   4
"optimizer":     AdamP
"precision":     16
"pretrained":    True
"scheduler":     {'OneCycleLR': {'total_steps': 20, 'max_lr': 0.0003}}
"size":          380
"subset":        0
"train_batches": 1.0
"train_trans":   Compose([
  RandomResizedCrop(always_apply=False, p=1, height=380, width=380, scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=2),
  Rotate(always_apply=False, p=0.5, limit=(-90, 90), interpolation=1, border_mode=4, value=None, mask_value=None),
  Flip(always_apply=False, p=0.5),
  Affine(always_apply=False, p=0.5, interpolation=1, cval=0, mode=4, scale={'x': (0.9, 1.1), 'y': (0.9, 1.1)}, translate_percent={'x': (-0.1, 0.1), 'y': (-0.1, 0.1)}, translate_px=None, rotate=(-15, 15), fit_output=False, shear={'x': (-10, 10), 'y': (-10, 10)}, cval_mask=0),
  Transpose(always_apply=False, p=0.5),

In [ ]:
!mkdir gram_ood
!mv gram-ood/* gram_ood/
!rm -r gram-ood

mkdir: cannot create directory ‘gram_ood’: File exists
mv: cannot stat 'gram-ood/*': No such file or directory
rm: cannot remove 'gram-ood': No such file or directory


# Setting the model

In [ ]:
from __future__ import division,print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
from tqdm.notebook import tqdm
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import gram_ood.utils.calculate_log as callog
from gram_ood.my_models import resnet_50
import gram_ood.utils.build_dataset as bd
import pandas as pd

In [ ]:
method = "gram-ood*" # or 'gram-ood'
# torch_model = resnet_50.Net(models.resnet50(pretrained=False), 8)
# ckpt = torch.load("gram_ood/checkpoints/resnet-50_checkpoint.pth")
# torch_model.load_state_dict(ckpt['model_state_dict'])
# torch_model.eval()
# torch_model.cuda()
print("Done!")

Done!


In [ ]:
from torchvision.models.resnet import Bottleneck
hooked_layers = list()
for layer in torch_model.modules():
  if method == 'gram-ood*':
    print(layer)
    if isinstance(layer, Bottleneck):
      hooked_layers.append(layer)
  else:
    if isinstance(layer, nn.ReLU) or isinstance(layer, nn.Conv2d):
      hooked_layers.append(layer)
hooked_layers

Se truncaron las últimas líneas 5000 del resultado de transmisión.
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(1056, 44, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(44, 1056, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pwl): Conv2d(1056, 176, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNormAct2d(
          176, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
        (drop_path): Identity()
      )
      (2): InvertedResidual(
        (conv_pw): Conv2d(176, 1056, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          1056, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
 

[]

## Setting the hook register

In [ ]:
feat_maps = list()
def _hook_fn(self, input, output):
    feat_maps.append(output)
    

def hook_layers(model):
    hooked_layers = list()
    for layer in torch_model.modules():
        if method == 'gram-ood*':
            #if isinstance(layer, models.resnet.Bottleneck):
            if isinstance(layer, nn.Conv2d):
                hooked_layers.append(layer)
        else:
            if isinstance(layer, nn.ReLU) or isinstance(layer, nn.Conv2d):
                hooked_layers.append(layer)
                
    return hooked_layers


def register_layers(layers):
    regs_layers = list()
    for lay in layers:
        regs_layers.append(lay.register_forward_hook(_hook_fn))
    return regs_layers


def unregister_layers(reg_layers):
    for lay in reg_layers:
        lay.remove()
                    

def get_feat_maps(model, batch_img):
    batch_img = batch_img.cuda()
    with torch.no_grad():
        preds = model(batch_img)

    preds = F.softmax(preds, dim=1)
    maps = feat_maps.copy()
    feat_maps.clear()
    return preds, maps

## Setting the hook
hl = hook_layers (torch_model)
rgl = register_layers (hl)
print ("Total number of registered hooked layers:", len(rgl))

Total number of registered hooked layers: 160


# Loading the data

In [ ]:
batch_ = 16

In [ ]:
test_csv = pd.read_csv("/content/drive/MyDrive/ISIC/2019/ISIC_2019_Test_Metadata.csv")
final_test = get_data(test_csv, "data/final_test/img", ".jpeg")

NameError: ignored

In [ ]:
!mkdir /content/gram_ood/data/skin_cancer/test_original
!mkdir /content/gram_ood/data/skin_cancer/test_original/img
!unzip /content/drive/MyDrive/ISIC/2019/ISIC_2019_Test_Input.zip -d /content/gram_ood/data/skin_cancer/test_original/
!mv /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/* /content/gram_ood/data/skin_cancer/test_original/img/
!rm -r /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input
!rm /content/gram_ood/data/skin_cancer/test_original/ATTRIBUTION.txt && rm /content/gram_ood/data/skin_cancer/test_original/LICENSE.txt


Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057954.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057955.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057956.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057957.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057961.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057965.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057967.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057968.jpg  
  inflating: /content/gram_ood/data/skin_cancer/test_original/ISIC_2019_Test_Input/ISIC_0057971.jpg  
  inflating: /c

In [ ]:
!mkdir /content/gram_ood/data/skin_cancer/test_original/img
!mv /content/gram_ood/data/skin_cancer/img/* /content/gram_ood/data/skin_cancer/test_original/img

## In distributions

In [ ]:
batch_size = batch_
trans = transforms.Compose([
            transforms.Resize((456,456)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])


sk_train = torch.utils.data.DataLoader(
                datasets.ImageFolder("/content/gram_ood/data/skin_cancer/train/",transform=trans),
                batch_size=batch_size,
                shuffle=False)

sk_test = torch.utils.data.DataLoader(
                datasets.ImageFolder("/content/gram_ood/data/skin_cancer/test_original/",transform=trans),
                batch_size=batch_size,
                shuffle=False)

## Out-of-distributions

In [ ]:
TRANS = transforms.Compose([
            transforms.Resize((456,456)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

PARAMS = {
    'batch_size': batch_,
    'shuf': False
}


def get_data (df, base_path, img_ext):    
    _imgs_path = df['image'].values
    imgs_path = [os.path.join(base_path, i + img_ext) for i in _imgs_path]
    dl = bd.get_data_loader(imgs_path, None, transform=TRANS, params=PARAMS)
    return dl 

# Gram-Matrix operations

## Gram matrix operator

In [ ]:
def norm_min_max(x):
    ma = torch.max(x,dim=1)[0].unsqueeze(1)
    mi = torch.min(x,dim=1)[0].unsqueeze(1)
    x = (x-mi)/(ma-mi)
    return x

def get_sims_gram_matrix (maps, power):
    maps = F.relu(maps)    
    maps = maps ** power    
    maps = maps.reshape(maps.shape[0],maps.shape[1],-1)
    gram = ((torch.matmul(maps,maps.transpose(dim0=2,dim1=1)))).sum(2)
    gram = (gram.sign()*torch.abs(gram)**(1/power)).reshape(gram.shape[0],-1)  
    
    if method == 'gram-ood*':
        gram = norm_min_max(gram)
        
    return gram


## Considering samples per label

In [ ]:
def _get_sim_per_labels(data_loader, power, use_preds=True):
    
    sims_per_label = None
    if not isinstance(power, list) and not isinstance(power, range):
        power = [power]
    
    for data in tqdm(data_loader):
        img_batch, labels = data 
        preds, maps_list = get_feat_maps(torch_model, img_batch)
      
        if use_preds:
            labels = preds.argmax(dim=1)  
                
        if sims_per_label is None:
            sims_per_label = [[[] for _ in range(len(maps_list))] for _ in range(preds.shape[1])]  
           
        for layer, maps in enumerate(maps_list): 
            for p in power:
                sims = get_sims_gram_matrix (maps, p)

                for sim, lab in zip(sims, labels):              
                    sims_per_label[lab.item()][layer].append(sim.cpu()) 
                
    return sims_per_label


def get_min_max_per_label(data_loader, power):
    
    sims_per_label = _get_sim_per_labels(data_loader, power)
    sims_per_label_min = [[[] for _ in range(len(sims_per_label[0]))] for _ in range(len(sims_per_label))] 
    sims_per_label_max = [[[] for _ in range(len(sims_per_label[0]))] for _ in range(len(sims_per_label))] 
    
    
    print ("-- Computing the values...")
    for lab_idx in range(len(sims_per_label)):
        for layer_idx in range(len(sims_per_label[lab_idx])):
            temp = torch.stack(sims_per_label[lab_idx][layer_idx])
            sims_per_label_min[lab_idx][layer_idx] = temp.min(dim=0)[0] 
            sims_per_label_max[lab_idx][layer_idx] = temp.max(dim=0)[0]
    
    del sims_per_label
    
    return sims_per_label_min, sims_per_label_max


def get_dev_scores_per_label_and_name(data_loader, power, sims_min, sims_max, ep=10e-6):
    
    if not isinstance(power, list) and not isinstance(power, range):
        power = [power]
    
    dev_scores = list()   
    img_names = list()
    for data in tqdm(data_loader):
        img_batch, _, _, img_name = data 
        preds_batch, maps_list = get_feat_maps(torch_model, img_batch)                
        labels = preds_batch.argmax(dim=1)
        batch_scores = list()
       
        for layer, maps in enumerate(maps_list):
                
            score_layer = 0
            for p in power:
                sims = get_sims_gram_matrix (maps, p)  
                _sim_min = torch.zeros(sims.shape[0], sims.shape[1]).cuda()
                _sim_max = torch.zeros(sims.shape[0], sims.shape[1]).cuda()
            
                for k, lab in enumerate(labels):
                    _sim_min[k] = sims_min[lab.item()][layer]
                    _sim_max[k] = sims_max[lab.item()][layer]            
            
                score_layer += (F.relu(_sim_min-sims)/torch.abs(_sim_min+ep)).sum(dim=1, keepdim=True)
                score_layer += (F.relu(sims-_sim_max)/torch.abs(_sim_max+ep)).sum(dim=1, keepdim=True)
           
            batch_scores.append(score_layer)            
            
        dev_scores.append(torch.cat(batch_scores, dim=1)) 
        img_names.append(img_name) 

    return torch.cat(dev_scores).cpu().numpy(), img_names


def get_dev_scores_per_label(data_loader, power, sims_min, sims_max, ep=10e-6):
    
    if not isinstance(power, list) and not isinstance(power, range):
        power = [power]
    
    dev_scores = list()   
    for data in tqdm(data_loader):
        img_batch, _ = data 
        preds_batch, maps_list = get_feat_maps(torch_model, img_batch)                
        labels = preds_batch.argmax(dim=1)
        batch_scores = list()
       
        for layer, maps in enumerate(maps_list):
                
            score_layer = 0
            for p in power:
                sims = get_sims_gram_matrix (maps, p)  
                _sim_min = torch.zeros(sims.shape[0], sims.shape[1]).cuda()
                _sim_max = torch.zeros(sims.shape[0], sims.shape[1]).cuda()
            
                for k, lab in enumerate(labels):
                    _sim_min[k] = sims_min[lab.item()][layer]
                    _sim_max[k] = sims_max[lab.item()][layer]            
            
                score_layer += (F.relu(_sim_min-sims)/torch.abs(_sim_min+ep)).sum(dim=1, keepdim=True)
                score_layer += (F.relu(sims-_sim_max)/torch.abs(_sim_max+ep)).sum(dim=1, keepdim=True)
           
            batch_scores.append(score_layer)            
            
        dev_scores.append(torch.cat(batch_scores, dim=1)) 

    return torch.cat(dev_scores).cpu().numpy()

In [ ]:
def detect_mean(all_test_std, all_ood_std, gaps=None): 
    
    avg_results = dict()
    indices = list(range(len(all_test_std)))
    split = int(np.floor(0.1 * len(all_test_std))) 
    for i in range(1,11):
        np.random.seed(i)
        np.random.shuffle(indices)
        
        val_std = all_test_std[indices[:split]]
        test_std = all_test_std[indices[split:]]
        
        if gaps is not None:
            t95 = (val_std.sum(axis=0) + gaps.mean(0))
        else:
            t95 = val_std.mean(axis=0) + 10**-7
        
        test_std = ((test_std)/t95[np.newaxis,:]).sum(axis=1)
        ood_std = ((all_ood_std)/t95[np.newaxis,:]).sum(axis=1)

        results = callog.compute_metric(-test_std,-ood_std)  

        for m in results:
            avg_results[m] = avg_results.get(m,0)+results[m]
    
    for m in avg_results:
        avg_results[m] /= i
        
        
    callog.print_results(avg_results)
    
    return avg_results

In [ ]:
def detect(all_test_std, all_ood_std):     
    
    indices = list(range(len(all_test_std)))
    split = int(np.floor(0.1 * len(all_test_std))) 
    np.random.seed(10)
    np.random.shuffle(indices)
        
    val_std = all_test_std[indices[:split]]
    test_std = all_test_std[indices[split:]]
        
    t95 = val_std.mean(axis=0) + 10**-7
        
    test_std = ((test_std)/t95[np.newaxis,:]).sum(axis=1)
    ood_std = ((all_ood_std)/t95[np.newaxis,:]).sum(axis=1)

    results = callog.compute_metric(-test_std,-ood_std)  

    callog.print_results(results)
    
    return results, ood_std

# OOD detection per label

In [ ]:
if method == 'gram-ood*':
    power = 1
else:
    power = range(1,10)

print ("- Getting mins/maxs")
mins, maxs = get_min_max_per_label(sk_train, power)

print ("- Getting test stdevs")
sk_test_stdev = get_dev_scores_per_label(sk_test, power, mins, maxs)

- Getting mins/maxs


  0%|          | 0/1267 [00:00<?, ?it/s]

In [ ]:
# Releasing the GPU memory
torch.cuda.empty_cache()

# Testing

In [ ]:
!cp -r gram_ood/data ./

In [ ]:
# efficientnet_b5
print("Final test new")
final_test_stdev, names = get_dev_scores_per_label_and_name(final_test, power, mins, maxs)
final_test_results, final = detect(sk_test_stdev, final_test_stdev)

In [ ]:
# resnetxt
print("Final test new")
final_test_stdev, names = get_dev_scores_per_label_and_name(final_test, power, mins, maxs)
final_test_results, final = detect(sk_test_stdev, final_test_stdev)

Final test new


  0%|          | 0/197 [00:00<?, ?it/s]

 TNR    AUROC  DTACC  AUIN   AUOUT 
  9.226 59.414 57.083 26.793 83.205


In [ ]:
print("Final test new")
final_test_stdev, names = get_dev_scores_per_label_and_name(final_test, power, mins, maxs)
final_test_results, final = detect(sk_test_stdev, final_test_stdev)

In [ ]:
print("Final test")
final_test_stdev, names = get_dev_scores_per_label_and_name(final_test, power, mins, maxs)
final_test_results, final = detect(sk_test_stdev, final_test_stdev)

Final test



 TNR    AUROC  DTACC  AUIN   AUOUT 
  7.028 45.456 51.311 18.628 78.163


## Saving deviations and image names

In [ ]:
!mkdir -p gram_ood/results/gram-ood*/

In [ ]:
np.savetxt('gram_ood/results/{}/final_resnet'.format(method), final_test_stdev.sum(axis=1), fmt='%.3f')
np.savetxt('gram_ood/results/{}/test_resnet'.format(method), sk_test_stdev.sum(axis=1), fmt='%.3f')

In [ ]:
clean_names = list()
for sub in names:
    for s in sub:
        clean_names.append(s)

np.savetxt('gram_ood/results/{}/names_resnet'.format(method), clean_names, fmt="%s")

# Merge

In [ ]:
method = 'gram-ood*' # or 'gram-ood'
model = 'efficientnet_b4'+ model_name # or 'resnet', 'densenet', 'mobilenet'

devs = np.loadtxt('gram_ood/results/{}/final_{}'.format(method, model))
img_names = np.genfromtxt('gram_ood/results/{}/names_{}'.format(method, model), dtype='str')
devs_norm = (devs - devs.min()) / (devs.max() - devs.min())
dic_names_devs = {x:y for x,y in zip(img_names, devs_norm)}

# predictions = pd.read_csv('gram_ood/isic_submit/merge/predictions.csv')
predictions = pd.read_csv('/content/drive/MyDrive/ISIC/2019/submissions/baseline-24/ensamble-baseline-24.csv')

cols = ['image', 'MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']
vals = list()        

print("Building the submission...")
for _, row in predictions.iterrows():
    name = row['image']
    new_unk = dic_names_devs[name]
    new_row = row[cols].values
    new_row[-1] = new_unk    
    vals.append(new_row)

new_pred = pd.DataFrame(vals, columns=cols)
# new_pred.to_csv("gram_ood/results/{}_{}.csv".format(method, model), index=False)
new_pred.to_csv("/content/drive/MyDrive/ISIC/2019/submissions/ood3/ensemble_ood.csv", index=False)
# new_pred.to_csv("/content/drive/MyDrive/ISIC/2019/submissions/ood2/unk.csv", index=False)

print("Done!")

Building the submission...
Done!


In [ ]:
new_pred

In [ ]:
!mkdir -p /content/drive/MyDrive/ISIC/2019/gram-ood*

In [ ]:
!cp results/gram-ood/* /content/drive/MyDrive/ISIC/2019/submissions/ood2/

In [ ]:
!ls /content/drive/MyDrive/ISIC/2019/gram-ood*/

final_resnet	  names_resnet	    test_resnet      unk.csv
final_resnet.csv  names_resnet.csv  test_resnet.csv


In [ ]:
subset = new_pred[["MEL", "NV", "BCC", "AK", "BKL", "DF", "VASC", "SCC", "UNK"]]
new_pred['OOD'] = subset.idxmax(axis=1).to_frame()
new_pred

In [ ]:
new_pred['OOD'] = new_pred.apply(lambda x: 1.0 if x['OOD']=='UNK' else 0.0, axis=1)
new_pred

In [ ]:
new_pred[new_pred['OOD']==1.0]

In [ ]:
ensemble = pd.read_csv('/content/drive/MyDrive/ISIC/2019/submissions/baseline-24/ensamble-baseline-24.csv')
ensemble['UNK'] = new_pred['OOD']
ensemble

ensemble.to_csv(f'/content/drive/MyDrive/ISIC/2019/submissions/ood2/ood-ensemble-b5.csv', index=False)

In [ ]:
ensemble = pd.read_csv('/content/drive/MyDrive/ISIC/2019/submissions/baseline-24/ensamble-baseline-24.csv')
ensemble.to_csv("/content/drive/MyDrive/ISIC/2019/submissions/ood/ensamble-baseline-24.csv", index=False)

In [ ]:
from glob import glob
import os
sub_dir = '/content/drive/MyDrive/ISIC/2019/submissions/ood/'
sorted(glob(os.path.join(sub_dir, '*csv')))

['/content/drive/MyDrive/ISIC/2019/submissions/ood/ensamble-baseline-24.csv',
 '/content/drive/MyDrive/ISIC/2019/submissions/ood/unk.csv']

In [ ]:
import pandas as pd
import numpy as np
subs = [pd.read_csv(csv) for csv in sorted(glob(os.path.join(sub_dir, '*csv')))]
  
wts = [1/(len(subs))]*(len(subs))
# wts = [0.1, 0.1 , 0.1, 0.1 , 0.1, 0.1 , 0.1, 0.2 , 0.05]
# wts = [1]*(len(subs))

AK = [sub.AK for sub in subs]
BCC = [sub.BCC for sub in subs]
BKL = [sub.BKL for sub in subs]
DF = [sub.DF for sub in subs]
MEL = [sub.MEL for sub in subs]
NV = [sub.NV for sub in subs]
SCC = [sub.SCC for sub in subs]
UNK = [sub.UNK for sub in subs]
VASC = [sub.VASC for sub in subs]



AK = np.sum([wts[i]*AK[i] for i in range(len(wts))],axis=0)
BCC = np.sum([wts[i]*BCC[i] for i in range(len(wts))],axis=0)
BKL = np.sum([wts[i]*BKL[i] for i in range(len(wts))],axis=0)
DF = np.sum([wts[i]*DF[i] for i in range(len(wts))],axis=0)
MEL = np.sum([wts[i]*MEL[i] for i in range(len(wts))],axis=0)
NV = np.sum([wts[i]*NV[i] for i in range(len(wts))],axis=0)
SCC = np.sum([wts[i]*SCC[i] for i in range(len(wts))],axis=0)
UNK = np.sum([wts[i]*UNK[i] for i in range(len(wts))],axis=0)
VASC = np.sum([wts[i]*VASC[i] for i in range(len(wts))],axis=0)

subs_prob = pd.DataFrame({'image': subs[0].image, 'AK': AK, 'BCC': BCC, 'BKL': BKL, 'DF': DF, 'MEL': MEL, 'NV': NV, 'SCC': SCC, 'UNK': UNK, 'VASC': VASC})

ensamble_name = 'ensamble-ood' 
subs_prob.to_csv(f'/content/drive/MyDrive/ISIC/2019/submissions/ood/{ensamble_name}.csv', index=False)